In [1]:
import joblib
import pandas as pd
from pathlib import Path
import sys

# project root (notebooks is inside project folder)
PROJECT_ROOT = Path(".").resolve().parent
sys.path.append(str(PROJECT_ROOT))

from utils.features import add_engineered_features


In [2]:
MODELS_DIR = PROJECT_ROOT / "models"

model_hours = joblib.load(MODELS_DIR / "model_hours.pkl")
model_crew = joblib.load(MODELS_DIR / "model_crew.pkl")

print("Models loaded successfully.")


Models loaded successfully.


In [3]:
sample_input = {
    "job_size_sqft": 1500,
    "num_rooms": 4,
    "num_heavy_items": 6,
    "num_light_items": 35,
    "distance_km": 15,
    "floor_number": 2,
    "has_elevator": True,
    "past_avg_hours": 6.5,
    "past_avg_crew_size": 3.0
}

X = pd.DataFrame([sample_input])
X_fe = add_engineered_features(X)
X_fe


,job_size_sqft,num_rooms,num_heavy_items,num_light_items,distance_km,floor_number,has_elevator,past_avg_hours,past_avg_crew_size,heavy_items_per_room,light_items_per_room,sqft_per_room,floor_penalty,elevator_relief
0,1500,4,6,35,15,2,1,6.5,3.0,1.2,7.0,300.0,0,0


In [4]:
pred_hours = float(model_hours.predict(X_fe)[0])
pred_crew = int(round(model_crew.predict(X_fe)[0]))

print("Predicted crew size:", pred_crew)
print("Predicted hours required:", pred_hours)



Predicted crew size: 3
Predicted hours required: 4.971666903786028


In [5]:
test_data = pd.DataFrame([
    {"job_size_sqft": 900, "num_rooms": 2, "num_heavy_items": 3, "num_light_items": 18,
     "distance_km": 5, "floor_number": 1, "has_elevator": True, "past_avg_hours": 3.5, "past_avg_crew_size": 2},
    {"job_size_sqft": 2200, "num_rooms": 6, "num_heavy_items": 10, "num_light_items": 60,
     "distance_km": 25, "floor_number": 3, "has_elevator": False, "past_avg_hours": 8, "past_avg_crew_size": 4},
    {"job_size_sqft": 3200, "num_rooms": 8, "num_heavy_items": 18, "num_light_items": 90,
     "distance_km": 40, "floor_number": 5, "has_elevator": False, "past_avg_hours": 10, "past_avg_crew_size": 5}
])

test_data_fe = add_engineered_features(test_data)

pred_hours = model_hours.predict(test_data_fe)
pred_crew = model_crew.predict(test_data_fe).round().astype(int)

test_data["predicted_crew_size"] = pred_crew
test_data["predicted_hours_required"] = pred_hours

test_data[["job_size_sqft", "predicted_crew_size", "predicted_hours_required"]]


,job_size_sqft,predicted_crew_size,predicted_hours_required
0,900,2,2.762156
1,2200,4,5.893157
2,3200,4,6.200283
